In [54]:
import os 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser

In [2]:
embedding_model = OllamaEmbeddings(model="nomic-embed-text:latest")

C:\Users\mahat\AppData\Local\Temp\ipykernel_19756\852141818.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="nomic-embed-text:latest")


In [8]:
urls = [
    "https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-2-reflection/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io"
]

In [17]:
docs = [WebBaseLoader(url).load() for url in urls]
doc_list = [item for doc in docs for item in doc]
doc_list

[Document(metadata={'source': 'https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io', 'title': 'Four AI Agent Strategies That Improve GPT-4 and GPT-3.5 Performance', 'description': 'I think AI agent workflows will drive massive AI progress this year — perhaps even more than the next generation of foundation models. This is an important...', 'language': 'en'}, page_content='Four AI Agent Strategies That Improve GPT-4 and GPT-3.5 Performance✨ New course! Enroll in  Event-Driven Agentic Document WorkflowsExplore CoursesAI NewsletterThe BatchAndrew\'s LetterData PointsML ResearchBlog✨ AI Dev 25CommunityForumEventsAmbassadorsAmbassador SpotlightResourcesCompanyAboutCareersContactStart LearningWeekly IssuesAndrew\'s LettersData PointsML ResearchBusinessScienceCultureHardwareAI CareersAboutSubscribeThe BatchLettersArticleAgentic Design Patterns Part 1 Four AI agent strategies that improve GPT-4 and GPT-3.5 performanceLettersTechnical In

In [22]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,
    chunk_overlap=0
)

doc_splits = text_splitter.split_documents(doc_list)

In [23]:
doc_splits

[Document(metadata={'source': 'https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io', 'title': 'Four AI Agent Strategies That Improve GPT-4 and GPT-3.5 Performance', 'description': 'I think AI agent workflows will drive massive AI progress this year — perhaps even more than the next generation of foundation models. This is an important...', 'language': 'en'}, page_content="Four AI Agent Strategies That Improve GPT-4 and GPT-3.5 Performance✨ New course! Enroll in  Event-Driven Agentic Document WorkflowsExplore CoursesAI NewsletterThe BatchAndrew's LetterData PointsML ResearchBlog✨ AI Dev 25CommunityForumEventsAmbassadorsAmbassador SpotlightResourcesCompanyAboutCareersContactStart LearningWeekly IssuesAndrew's LettersData PointsML ResearchBusinessScienceCultureHardwareAI CareersAboutSubscribeThe BatchLettersArticleAgentic Design Patterns Part 1 Four AI agent strategies that improve GPT-4 and GPT-3.5 performanceLettersTechnical Insi

In [39]:
vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=embedding_model,
)

In [40]:
retriver = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [42]:
question = "what are the differnt kind of agentic design patterns?"
ans = retriver.invoke(question)
ans

[Document(id='5ef1c25d-805b-45f3-89ad-46233cf1ac3c', metadata={'source': 'https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io', 'title': 'Agentic Design Patterns Part 5, Multi-Agent Collaboration', 'description': 'Multi-agent collaboration is the last of the four\xa0key AI agentic design patterns\xa0that I’ve described in recent letters...', 'language': 'en'}, page_content='clear, efficient code as opposed to, say, scalable and highly secure code. By decomposing the overall task into subtasks, we can optimize the subtasks better.Perhaps most important, the multi-agent design pattern gives us, as developers, a framework for breaking down complex tasks into subtasks. When writing code to run on a single CPU, we often break our program up into different processes or threads. This is a useful abstraction that lets us decompose a task, like implementing a web browser, into subtasks that are easier to code. I find th

In [44]:
print(ans[0].page_content)

clear, efficient code as opposed to, say, scalable and highly secure code. By decomposing the overall task into subtasks, we can optimize the subtasks better.Perhaps most important, the multi-agent design pattern gives us, as developers, a framework for breaking down complex tasks into subtasks. When writing code to run on a single CPU, we often break our program up into different processes or threads. This is a useful abstraction that lets us decompose a task, like implementing a web browser, into subtasks that are easier to code. I find thinking through multi-agent roles to be a useful abstraction as well.In many companies, managers routinely decide what roles to hire, and then how to split complex projects — like writing a large piece of software or preparing a research report — into smaller tasks to assign to employees with different specialties. Using multiple agents is analogous. Each agent implements its own workflow, has its own memory (itself a rapidly evolving area in agentic

In [55]:
llm = OllamaLLM(model="qwen2.5:3b")
system = """You are a grader assessing the relevance of a retrieved document to a user question.
If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant.
It does not need to be a stringent test. The goal is to filter out erroneous retrievals.
Reply with either 'yes' or 'no' only."""

# Define the prompt template
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# Define output parser to extract structured output
parser = StrOutputParser()

# Combine prompt, LLM, and parser
retrieval_grader = grade_prompt | llm | parser

In [60]:
doc_to_use = []

for doc in ans:
    print(doc.page_content,'\n','-'*100)
    res = retrieval_grader.invoke({"question":question,"document":doc.page_content})
    print(res)
    if res=='yes':
        doc_to_use.append(doc)

clear, efficient code as opposed to, say, scalable and highly secure code. By decomposing the overall task into subtasks, we can optimize the subtasks better.Perhaps most important, the multi-agent design pattern gives us, as developers, a framework for breaking down complex tasks into subtasks. When writing code to run on a single CPU, we often break our program up into different processes or threads. This is a useful abstraction that lets us decompose a task, like implementing a web browser, into subtasks that are easier to code. I find thinking through multi-agent roles to be a useful abstraction as well.In many companies, managers routinely decide what roles to hire, and then how to split complex projects — like writing a large piece of software or preparing a research report — into smaller tasks to assign to employees with different specialties. Using multiple agents is analogous. Each agent implements its own workflow, has its own memory (itself a rapidly evolving area in agentic

In [61]:
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. 
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

# Post-processing
def format_docs(docs):
    return "\n".join(f"<doc{i+1}>:\nTitle:{doc.metadata['title']}\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i+1}>\n" for i, doc in enumerate(docs))

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"documents":format_docs(doc_to_use), "question": question})
print(generation)

There are four main kinds of agentic design patterns: Reflection, Tool Use, Planning, and Multi-Agent Collaboration. Reflection involves the AI examining its own work to improve it. Tool Use is when the AI uses external tools like web searches or code execution. Planning refers to creating and executing a multi-step plan to achieve a goal. Lastly, Multi-Agent Collaboration involves multiple AI agents working together to solve problems better than a single agent could.
